# Classification of Sugarcane Diseases based on Images

## Initial Setup

Examining the train data shows that there are six (6) classes in total:

In [16]:
classes = [
    "Banded_Chlorosis",
    "Brown_Rust",
    "Brown_Spot",
    "Viral",
    "Yellow_Leaf",
    "Healthy",
]

In [ ]:
import os
from pprint import pprint

import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

BATCH_SIZE: int = 64
IMAGE_SIZE: int = 128
SEED: int = 1738
DATASET_PATH: str = "train"

class_distribution = {}

for class_name in classes:
    class_distribution[class_name] = len(os.listdir(f"{DATASET_PATH}/{class_name}"))

minimum_class_count = min(class_distribution.values())

dataset: tf.data.Dataset = keras.utils.image_dataset_from_directory(
    directory=DATASET_PATH,
    shuffle=True,
    labels="inferred",
    label_mode="categorical",
    class_names=classes,
    color_mode="rgb",
    batch_size=None,  # type: ignore
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    pad_to_aspect_ratio=True)

def normalize(image: tf.Tensor, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(normalize)

# Check if normalized
# image, label = list(dataset.take(1).as_numpy_iterator())[0]
# print(plt.imshow(image.astype('uint8')))
# print(image.max(), image.min())

class_datasets = []
for class_index, class_name in enumerate(classes):
    class_dataset = dataset.filter(
        lambda image, label: tf.argmax(label) == class_index
    )
    class_datasets.append(class_dataset.take(minimum_class_count))  # limit classes to minimum class count

weights = [1.0 / len(classes)] * len(classes) # equal weights
balanced_dataset = tf.data.experimental.sample_from_datasets(
    class_datasets, weights=weights, seed=SEED
)
balanced_dataset = (
    balanced_dataset.shuffle(1000, seed=SEED)
    .batch(BATCH_SIZE)
    # .prefetch(tf.data.AUTOTUNE)
)

# Verify the class distribution
class_counts = {class_name: 0 for class_name in classes}
for _, label_batch in balanced_dataset.take(10):  #  type: ignore
    for label in label_batch:
        class_index = tf.argmax(label).numpy()
        class_counts[classes[class_index]] += 1

print("Class distribution in balanced_dataset:")
pprint(class_counts)

Found 4314 files belonging to 6 classes.
0.92156863 0.0
Class distribution in balanced_dataset:
{'Banded_Chlorosis': 108,
 'Brown_Rust': 103,
 'Brown_Spot': 98,
 'Healthy': 90,
 'Viral': 112,
 'Yellow_Leaf': 129}


In [147]:
VALIDATION_SPLIT = 0.2

# Calculate the total number of samples in the dataset
total_samples = sum(1 for _ in balanced_dataset)

# Calculate the number of samples for training and validation
train_size = int(total_samples * (1-VALIDATION_SPLIT))
val_size = total_samples - train_size

# Split the dataset
train_dataset = balanced_dataset.take(train_size)  # Take the first `train_size` samples
val_dataset = balanced_dataset.skip(train_size)    # Skip the first `train_size` samples


## Data Loading and Preprocessing

First, create the instances for each class.

## Method 1: Convolution Neural Network (CNN)

The first method for solving this classification problem is through the use of CNN.

We will be using the Keras and TensorFlow libraries.

First, let us setup the environment and model.

In [153]:

import keras

# Set Keras backend to use TensorFlow
os.environ["KERAS_BACKEND"] = "tensorflow"

DROPOUT_RATE = 0.20

cnn_model = keras.models.Sequential(
    [
        keras.layers.InputLayer(shape=(128, 128, 3)),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding="Same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="Same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=96, kernel_size=(3, 3), padding="Same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="Same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=192, kernel_size=(3, 3), padding="Same", activation="relu"),
        keras.layers.Dropout(DROPOUT_RATE),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="Same", activation="relu"),
        keras.layers.Dropout(DROPOUT_RATE),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        # fully connected
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dropout(DROPOUT_RATE),
        keras.layers.Dense(len(classes), activation="softmax"),
    ]
)

cnn_model.summary()


cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),  # type: ignore
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

EPOCHS = 5
total_samples = sum(1 for _ in train_dataset.unbatch())  # Unbatch to count individual samples

# Calculate steps_per_epoch
steps_per_epoch = total_samples // BATCH_SIZE

history = cnn_model.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_112 (Conv2D)             │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_113 (Conv2D)             │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_114 (Conv2D)             │ (None, 32, 32, 96)     │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 16, 16, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_115 (Conv2D)             │ (None, 16, 16, 128)    │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_116 (Conv2D)             │ (None, 8, 8, 192)      │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 8, 8, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 4, 4, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_117 (Conv2D)             │ (None, 4, 4, 256)      │       442,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,377,382 (5.25 MB)

 Trainable params: 1,377,382 (5.25 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 911ms/step - accuracy: 0.1674 - loss: 1.7960 - val_accuracy: 0.2356 - val_loss: 1.7774
Epoch 2/5


/home/ubuntu/CS180-Project/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 228ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2644 - val_loss: 1.7432
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 692ms/step - accuracy: 0.2662 - loss: 1.7849 - val_accuracy: 0.2615 - val_loss: 1.7031
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3190 - val_loss: 1.7014
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 738ms/step - accuracy: 0.3069 - loss: 1.6484 - val_accuracy: 0.3448 - val_loss: 1.6159


Displaying the accuracy and validation loss across epochs.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["val_loss"], color="b", label="validation loss")
plt.plot(history.history["loss"], color="r", label="accuracy loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Method 2: Transfer Learning from InceptionV3

In [ ]:
import keras
from keras.api.applications.inception_v3 import InceptionV3

EPOCHS = 50
BATCH_SIZE = 256

pre_trained_model = InceptionV3(
    input_shape=X_train.shape[1::],
    weights="imagenet",
    include_top=False,  # the fully connected layer at the end
)

for layer in pre_trained_model.layers:
    layer.trainable = False

fully_connected_layer = keras.layers.Flatten()(pre_trained_model.output)
fully_connected_layer = keras.layers.Dense(512, activation="relu")(fully_connected_layer)
fully_connected_layer = keras.layers.Dropout(0.2)(fully_connected_layer)
fully_connected_layer = keras.layers.Dense(len(classes), activation="softmax")(
    fully_connected_layer
)

transfer_model = keras.Model(pre_trained_model.input, fully_connected_layer)

# transfer_model.summary()

transfer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),  # type: ignore
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy(), "acc"],
)

history = transfer_model.fit(
    X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_test, Y_test)
)

In [ ]:
import matplotlib.pyplot as plt

# show the accuracy of method 2
plt.plot(history.history["acc"], color="b", label="accuracy")
plt.title("Accuracy of Transfer Model (Inceptionv3)")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# show the training (and validation) loss of method 2
plt.plot(history.history["val_loss"], color="b", label="validation loss")
plt.plot(history.history["loss"], color="r", label="accuracy loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
transfer_model.summary()